##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TFX と Vertex Pipelines を使った Vertex AI トレーニングとサービング


<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/gcp/vertex_pipelines_vertex_training"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tfx/tutorials/tfx/gcp/vertex_pipelines_vertex_training.ipynb"> <img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tfx/tutorials/tfx/gcp/vertex_pipelines_vertex_training.ipynb"> <img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHubでソースを表示</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/tfx/docs/tutorials/tfx/gcp/vertex_pipelines_vertex_training.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
<td><a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?q=download_url%3Dhttps://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tfx/tutorials/tfx/gcp/vertex_pipelines_vertex_training.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Google Cloud Vertex AI Workbench で実行</a></td>
</table></div>


ノートブックを使用したこのチュートリアルでは、Vertex AI Training サービスを使って ML モデルをトレーニングし、サービング目的で Vertex AI に公開する TFX パイプラインを作成して実行します。

このノートブックは、[Vertex Pipelines 用の単純な TFX パイプラインのチュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/gcp/vertex_pipelines_simple)を基礎としています。そのチュートリアルをまだお読みでない場合は、このノートブックを進める前に目を通しておくことをお勧めします。

モデルは AutoML を使って Vertex AI でトレーニングするか、カスタムトレーニングを使用することができます。カスタムトレーニングの場合は、トレーニングジョブの強化、分散トレーニングの有効化、ハイパーパラメータチューニングの使用、GPU によるアクセラレーションにおいて、多数の様々なマシンタイプの選択が可能です。

また、トレーニング済みのモデルを Vertex AI Models にデプロイし、エンドポイントを作成することで、予測リクエストをサービングすることもできます。

このチュートリアルでは、TFX パイプラインでモデルをトレーニングするカスタムジョブによって Vertex AI Training を使用します。また、Vertex AI を使って予測リクエストをサービングするモデルもデプロイします。


このノートブックは、[Google Colab](https://colab.research.google.com/notebooks/intro.ipynb) または [AI Platform Notebooks](https://cloud.google.com/ai-platform-notebooks) で実行することを意図しています。いずれも使用していない場合は、上記の「Google Colab で実行」をクリックしてください。

## セットアップ

[Vertex Pipelines 用の単純な TFX パイプラインのチュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/gcp/vertex_pipelines_simple)を完了した場合は、作業用の GCP プロジェクトと GCS バケットがすでに存在します。このチュートリアルで必要なのはこれだけです。そのチュートリアルを実行していない場合は、前準備のチュートリアルをお読みください。

### Python パッケージをインストールする

TFX や KFP など、ML パイプラインのオーサリングと Vertex Pipelines へのジョブの送信に必要な Python パッケージをインストールします。

In [ ]:
# Use the latest version of pip.
!pip install --upgrade pip
!pip install --upgrade "tfx[kfp]<2"

#### ランタイムを再起動しましたか？

Google Colab を使用している場合は、上記のセルを初めて実行した後に、「ランタイムを再起動」ボタンをクリックするか、「ランタイム」&gt;「ランタイムの再起動...」メニューを使って、ランタイムを再起動する必要があります。これは、Colab がパッケージを読み込むために必要な作業です。

Colab を使用していない場合は、以下のセルを使ってランタイムを再起動できます。

In [ ]:
# docs_infra: no_execute
import sys
if not 'google.colab' in sys.modules:
  # Automatically restart kernel after installs
  import IPython
  app = IPython.Application.instance()
  app.kernel.do_shutdown(True)

### このノートブックについて Google にログインする

このノートブックを Colab で実行している場合は、お使いのユーザーアカウントで認証を行います。

In [ ]:
import sys
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()

**AI Platform Notebooks を使用している場合**は、次のセクションを実行する前に、以下を実行して Google Cloud で認証を行います。

```sh
gcloud auth login
```

この認証は、**ターミナルウィンドウ**（**ファイル** &gt; **新規**）で行います。この作業は、ノートブックインスタンスごとに 1 回だけ必要です。

パッケージのバージョンを確認します。

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))
import kfp
print('KFP version: {}'.format(kfp.__version__))

### 変数をセットアップする

以下のパイプラインをカスタマイズするために使用する変数をいくつかセットアップします。以下の情報が必要です。

- GCP プロジェクト ID。[プロジェクト ID を特定する](https://cloud.google.com/resource-manager/docs/creating-managing-projects#identifying_projects)をご覧ください。
- パイプラインを実行する GCP リージョン。Vertex Pipelines を利用できるリージョンについての詳細は、[Vertex AI のロケーションガイド](https://cloud.google.com/vertex-ai/docs/general/locations#feature-availability)をご覧ください。
- パイプライン出力を保存する Google Cloud Storage バケット。

**以下のセルに必要な値を入力してから実行します**。


In [ ]:
GOOGLE_CLOUD_PROJECT = ''     # <--- ENTER THIS
GOOGLE_CLOUD_REGION = ''      # <--- ENTER THIS
GCS_BUCKET_NAME = ''          # <--- ENTER THIS

if not (GOOGLE_CLOUD_PROJECT and GOOGLE_CLOUD_REGION and GCS_BUCKET_NAME):
    from absl import logging
    logging.error('Please set all required parameters.')

`gcloud` を使用するプロジェクトに設定します。

In [ ]:
!gcloud config set project {GOOGLE_CLOUD_PROJECT}

In [ ]:
PIPELINE_NAME = 'penguin-vertex-training'

# Path to various pipeline artifact.
PIPELINE_ROOT = 'gs://{}/pipeline_root/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for users' Python module.
MODULE_ROOT = 'gs://{}/pipeline_module/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for users' data.
DATA_ROOT = 'gs://{}/data/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

# Name of Vertex AI Endpoint.
ENDPOINT_NAME = 'prediction-' + PIPELINE_NAME

print('PIPELINE_ROOT: {}'.format(PIPELINE_ROOT))

### サンプルデータを準備する

[単純な TFX パイプラインのチュートリアル](https://allisonhorst.github.io/palmerpenguins/articles/intro.html)と同じ [Palmer Penguins データセット](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple)を使用します。

このデータセットには、範囲 [0,1] を持つようにすでに正規化された数値特徴量が 4 つあります。ペンギンの `species`（種）を予測する分類モデルを構築します。

データセットのコピーを作成する必要があります。TFX ExampleGen はディレクトリから入力を読み取るため、ディレクトリを作成してデータセットをそれにコピーする必要があるためです。

In [ ]:
!gsutil cp gs://download.tensorflow.org/data/palmer_penguins/penguins_processed.csv {DATA_ROOT}/

CSV ファイルを見てみましょう。

In [ ]:
!gsutil cat {DATA_ROOT}/penguins_processed.csv | head

## パイプラインを作成する

このパイプラインは、[Vertex Pipelines 用の単純な TFX パイプラインのチュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/gcp/vertex_pipelines_simple)で作成したパイプラインによく似ています。このパイプラインは  CsvExampleGen、Trainer、Pusher という 3 つのコンポーネントで構成されますが、ここで使用する Trainer と Pusher コンポーネントは特殊です。Trainer コンポーネントはトレーニングワークロードを Vertex AI に移行し、Pusher コンポーネントはトレーニングした ML モデルを、ファイルシステムにではなく、Vertex AI に公開します。

TFX には、Vertex AI Training サービスにトレーニングジョブを送信する特殊な `Trainer` があります。ユーザー側では、標準の `Trainer` コンポーネントの代わりに拡張機能モジュールの `Trainer` を使用し、必要な GCP パラメータを設定する作業のみが必要です。

このチュートリアルでは、先に CPU のみで、その後に GPU も使って、Vertex AI Training ジョブを実行します。

TFX には、*Vertex AI Models* にモデルをアップロードする特殊な `Pusher` もあります。`Pusher` は、オンラインで予測をサービングするための *Vertex AI Endpoint* リソースも作成します。Vertex AI が提供するオンライン予測についての詳細は、[Vertex AI ドキュメント](https://cloud.google.com/vertex-ai/docs/predictions/getting-predictions)をご覧ください。

### モデルコードを記述する

モデル自体は、[単純な TFX パイプラインのチュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple)によく似ています。

[TensorFlow 分散ストラテジー](https://www.tensorflow.org/guide/distributed_training)を作る `_get_distribution_strategy()` 関数を追加し、GPU を使用できる場合には MirroedStrategy を使用できるように `run_fn` で使用されます。

In [ ]:
_trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

# Copied from https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple and
# slightly modified run_fn() to add distribution_strategy.

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_metadata.proto.v0 import schema_pb2
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

# Since we're not generating or creating a schema, we will instead create
# a feature spec.  Since there are a fairly small number of features this is
# manageable for this dataset.
_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
        for feature in _FEATURE_KEYS
    }, _LABEL_KEY: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}


def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _make_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# NEW: Read `use_gpu` from the custom_config of the Trainer.
#      if it uses GPU, enable MirroredStrategy.
def _get_distribution_strategy(fn_args: tfx.components.FnArgs):
  if fn_args.custom_config.get('use_gpu', False):
    logging.info('Using MirroredStrategy with one GPU.')
    return tf.distribute.MirroredStrategy(devices=['device:GPU:0'])
  return None


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # This schema is usually either an output of SchemaGen or a manually-curated
  # version provided by pipeline author. A schema can also derived from TFT
  # graph if a Transform component is used. In the case when either is missing,
  # `schema_from_feature_spec` could be used to generate schema from very simple
  # feature_spec, but the schema returned would be very primitive.
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  # NEW: If we have a distribution strategy, build a model in a strategy scope.
  strategy = _get_distribution_strategy(fn_args)
  if strategy is None:
    model = _make_keras_model()
  else:
    with strategy.scope():
      model = _make_keras_model()

  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

このモジュールファイルを、パイプラインコンポーネントからアクセスできる GCS にコピーします。

そうしない場合は、モジュールファイルを含むコンテナイメージをビルドし、そのイメージをパイプラインと AI Platform Training ジョブで実行することが可能です。

In [ ]:
!gsutil cp {_trainer_module_file} {MODULE_ROOT}/

### パイプライン定義を記述する

TFX パイプラインを作成する関数を定義します。これには、[単純な TFX パイプラインのチュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple)と同じ 3 つのコンポーネントが含まれますが、GCP 拡張機能モジュールの `Trainer` と `Pusher` コンポーネントを使用します。

`tfx.extensions.google_cloud_ai_platform.Trainer` は通常の `Trainer` のように動作しますが、モデルトレーニングの計算をクラウドに移動している点が異なります。Vertex AI Training サービスでカスタムジョブを起動し、オーケストレーションシステムの trainer コンポーネントは Vertex AI Training ジョブが完了するのを待機します。

`tfx.extensions.google_cloud_ai_platform.Pusher`  は、トレーニング済みのモデルを使って Vertex AI Model と Vertex AI Endpoint を作成します。


In [ ]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, endpoint_name: str, project_id: str,
                     region: str, use_gpu: bool) -> tfx.dsl.Pipeline:
  """Implements the penguin pipeline with TFX."""
  # Brings data into the pipeline or otherwise joins/converts training data.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # NEW: Configuration for Vertex AI Training.
  # This dictionary will be passed as `CustomJobSpec`.
  vertex_job_spec = {
      'project': project_id,
      'worker_pool_specs': [{
          'machine_spec': {
              'machine_type': 'n1-standard-4',
          },
          'replica_count': 1,
          'container_spec': {
              'image_uri': 'gcr.io/tfx-oss-public/tfx:{}'.format(tfx.__version__),
          },
      }],
  }
  if use_gpu:
    # See https://cloud.google.com/vertex-ai/docs/reference/rest/v1/MachineSpec#acceleratortype
    # for available machine types.
    vertex_job_spec['worker_pool_specs'][0]['machine_spec'].update({
        'accelerator_type': 'NVIDIA_TESLA_K80',
        'accelerator_count': 1
    })

  # Trains a model using Vertex AI Training.
  # NEW: We need to specify a Trainer for GCP with related configs.
  trainer = tfx.extensions.google_cloud_ai_platform.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5),
      custom_config={
          tfx.extensions.google_cloud_ai_platform.ENABLE_VERTEX_KEY:
              True,
          tfx.extensions.google_cloud_ai_platform.VERTEX_REGION_KEY:
              region,
          tfx.extensions.google_cloud_ai_platform.TRAINING_ARGS_KEY:
              vertex_job_spec,
          'use_gpu':
              use_gpu,
      })

  # NEW: Configuration for pusher.
  vertex_serving_spec = {
      'project_id': project_id,
      'endpoint_name': endpoint_name,
      # Remaining argument is passed to aiplatform.Model.deploy()
      # See https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api#deploy_the_model
      # for the detail.
      #
      # Machine type is the compute resource to serve prediction requests.
      # See https://cloud.google.com/vertex-ai/docs/predictions/configure-compute#machine-types
      # for available machine types and acccerators.
      'machine_type': 'n1-standard-4',
  }

  # Vertex AI provides pre-built containers with various configurations for
  # serving.
  # See https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers
  # for available container images.
  serving_image = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-6:latest'
  if use_gpu:
    vertex_serving_spec.update({
        'accelerator_type': 'NVIDIA_TESLA_K80',
        'accelerator_count': 1
    })
    serving_image = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-gpu.2-6:latest'

  # NEW: Pushes the model to Vertex AI.
  pusher = tfx.extensions.google_cloud_ai_platform.Pusher(
      model=trainer.outputs['model'],
      custom_config={
          tfx.extensions.google_cloud_ai_platform.ENABLE_VERTEX_KEY:
              True,
          tfx.extensions.google_cloud_ai_platform.VERTEX_REGION_KEY:
              region,
          tfx.extensions.google_cloud_ai_platform.VERTEX_CONTAINER_IMAGE_URI_KEY:
              serving_image,
          tfx.extensions.google_cloud_ai_platform.SERVING_ARGS_KEY:
            vertex_serving_spec,
      })

  components = [
      example_gen,
      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      components=components)

## Vertex Pipelines でパイプラインを実行する

[Vertex Pipelines 用の単純な TFX パイプラインのチュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/gcp/vertex_pipelines_simple)と同様に、Vertex Pipelines を使ってパイプラインを実行します。

In [ ]:
# docs_infra: no_execute
import os

PIPELINE_DEFINITION_FILE = PIPELINE_NAME + '_pipeline.json'

runner = tfx.orchestration.experimental.KubeflowV2DagRunner(
    config=tfx.orchestration.experimental.KubeflowV2DagRunnerConfig(),
    output_filename=PIPELINE_DEFINITION_FILE)
_ = runner.run(
    _create_pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=PIPELINE_ROOT,
        data_root=DATA_ROOT,
        module_file=os.path.join(MODULE_ROOT, _trainer_module_file),
        endpoint_name=ENDPOINT_NAME,
        project_id=GOOGLE_CLOUD_PROJECT,
        region=GOOGLE_CLOUD_REGION,
        # We will use CPUs only for now.
        use_gpu=False))

生成された定義ファイルは、`google-cloud-aiplatform` パッケージの Google Cloud aiplatform クライアントを使って送信できます。

In [ ]:
# docs_infra: no_execute
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs
import logging
logging.getLogger().setLevel(logging.INFO)

aiplatform.init(project=GOOGLE_CLOUD_PROJECT, location=GOOGLE_CLOUD_REGION)

job = pipeline_jobs.PipelineJob(template_path=PIPELINE_DEFINITION_FILE,
                                display_name=PIPELINE_NAME)
job.submit()

上記の出力に含まれるリンクにアクセスするか、[Google Cloud Console](https://console.cloud.google.com/) で［Vertex AI］&gt;［パイプライン］にアクセスすると、進捗状況を確認できます。

## 予測リクエストでテストする

パイプラインが完了したら、［Vertex AI］&gt;［エンドポイント］の 1 つに*デプロイされた*モデルを確認できます。新しいエンドポイントに予測リクエストを送信するには、そのエンドポイントの ID が必要です。これは、前に入力した*エンドポイント名*とは異なります。この ID は、`Google Cloud Console` の[エンドポイントページ](https://console.cloud.google.com/vertex-ai/endpoints)に記載されており、非常に長い番号です。

**以下に ENDPOINT_ID を設定してから実行します。**


In [ ]:
ENDPOINT_ID=''     # <--- ENTER THIS
if not ENDPOINT_ID:
    from absl import logging
    logging.error('Please set the endpoint id.')

同じ aiplatform クライアントを使用して、エンドポイントにリクエストを送信します。ここでは、ペンギンの種の分類を行うための予測リクエストを送信します。入力はここで使用した 4 つの特徴量で、モデルが返すのは 3 つの値です。これは、モデルが各 species に対して 1 つの値を出力するためです。

たとえば、以下の具体的な例には、インデックス 2 に最大値があり、「2」を出力します。


In [ ]:
# docs_infra: no_execute
import numpy as np

# The AI Platform services require regional API endpoints.
client_options = {
    'api_endpoint': GOOGLE_CLOUD_REGION + '-aiplatform.googleapis.com'
    }
# Initialize client that will be used to create and send requests.
client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)

# Set data values for the prediction request.
# Our model expects 4 feature inputs and produces 3 output values for each
# species. Note that the output is logit value rather than probabilities.
# See the model code to understand input / output structure.
instances = [{
    'culmen_length_mm':[0.71],
    'culmen_depth_mm':[0.38],
    'flipper_length_mm':[0.98],
    'body_mass_g': [0.78],
}]

endpoint = client.endpoint_path(
    project=GOOGLE_CLOUD_PROJECT,
    location=GOOGLE_CLOUD_REGION,
    endpoint=ENDPOINT_ID,
)
# Send a prediction request and get response.
response = client.predict(endpoint=endpoint, instances=instances)

# Uses argmax to find the index of the maximum value.
print('species:', np.argmax(response.predictions[0]))

オンライン予測についての詳細は、`Google Cloud Console` の[エンドポイントページ](https://console.cloud.google.com/vertex-ai/endpoints)をご覧ください。サンプルリクエストの送信ガイドとその他のリソースへのリンクが記載されています。

## GPU を使ってパイプラインを実行する


Vertex AI は、GPU のサポートを含み、様々なマシンタイプを使ったトレーニングをサポートしています。利用可能なオプションについては、[マシンの仕様に関するリファレンス](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/MachineSpec#acceleratortype)をご覧ください。

パイプラインは、GPU トレーニングをサポートするように定義済みです。後は、`use_gpu` フラグを True に設定すれば、準備完了です。すると、パイプラインは 1 つの NVIDIA_TESLA_K80 を含めたマシン仕様を作成し、モデルトレーニングコードでは `tf.distribute.MirroredStrategy` が使用されるようになります。

`use_gpu` フラグは Vertex または TFX API の一部ではないことに注意してください。このフラグは、このチュートリアルでトレーニングコードを制御するためだけに使用されています。

In [ ]:
# docs_infra: no_execute
runner.run(
    _create_pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=PIPELINE_ROOT,
        data_root=DATA_ROOT,
        module_file=os.path.join(MODULE_ROOT, _trainer_module_file),
        endpoint_name=ENDPOINT_NAME,
        project_id=GOOGLE_CLOUD_PROJECT,
        region=GOOGLE_CLOUD_REGION,
        # Updated: Use GPUs. We will use a NVIDIA_TESLA_K80 and 
        # the model code will use tf.distribute.MirroredStrategy.
        use_gpu=True))

job = pipeline_jobs.PipelineJob(template_path=PIPELINE_DEFINITION_FILE,
                                display_name=PIPELINE_NAME)
job.submit()

上記の出力に含まれるリンクにアクセスするか、[Google Cloud Console](https://console.cloud.google.com/) で［Vertex AI］&gt;［パイプライン］にアクセスすると、進捗状況を確認できます。

## クリーンアップ

このチュートリアルでは、Vertex AI Model と Endpoint を作成しました。不要な変更が行われないようにこれらのリソースを削除してください。[Endpoints](https://console.cloud.google.com/vertex-ai/endpoints) にアクセスし、先にエンドポイントからモデルを*アンデプロイ*してから、エンドポイントとモデルを個別に削除することができます。